In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# sample list
mapping_16S = pd.read_csv("~/Documents/thesis/16S/qiime2/mappingfiles/16S_mapping.csv")
mapping_16S.head()

,#SampleID,Mother_Child,SubjectID,MaternalID,TimePoint,Mgx_batch Mapping,16S_batch Mapping
0,C0052-5F-1A,C,52,0052_m,5,Mgx_batch001,16S_batch001
1,C0126-8F-1A,C,126,0126_m,8,Mgx_batch001,16S_batch001
2,C0126-8F-1B,C,126,0126_m,8,Mgx_batch001,16S_batch001
3,C0127-6F-1B,C,127,0127_m,6,Mgx_batch001,16S_batch001
4,C0286-5F-1A,C,286,0286_m,5,Mgx_batch001,16S_batch001


In [3]:
samplenames = list(mapping_16S["#SampleID"])

In [4]:
# read in mgx data
mgx = pd.read_csv("mgx_genera.csv")
mgx.rename(columns={'Taxon':'taxa'}, inplace=True)
mgx.head()


,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
0,Acidaminococcaceae_unclassified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
1,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.002632,0.0,0.0,0.000000,0.03501,0.0,0.0
2,Acinetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
3,Actinobacillus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000139,0.000000,0.0,0.0,0.000116,0.00000,0.0,0.0
4,Actinobaculum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [5]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
mgx = mgx[~mgx.taxa.str.contains("_unclassified")]
mgx = mgx[~mgx.taxa.str.contains("_noname")]

In [49]:
mgx.sample(15)

,taxa,M0652-1F-1A,M0647-1F-1A,M0656-1F-1A,C0660-1E-1A,M0649-1F-1A,C0683-1E-1A,C0686-1E-1A,M0647-2F-1A,M0647-2E-1A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
68,Escherichia,0.000000,0.000634,0.002915,0.000214,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001101,0.000000,0.000239,0.000164,0.000000,0.002176,0.000826,0.000342,0.002337,0.000000
86,Klebsiella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
143,Salmonella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
91,Leptotrichia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
160,Tunalikevirus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
23,Bifidobacterium,0.049129,0.046362,0.123536,0.041587,0.109875,0.014435,0.069332,0.100865,0.025837,...,0.049404,0.660008,0.115940,0.046643,0.000000,0.002417,0.040178,0.019396,0.038867,0.027026
163,Varibaculum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131,Proteus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26,Bocavirus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
83,Holdemania,0.000000,0.000000,0.000000,0.000000,0.000000,0.000029,0.000000,0.000193,0.000000,...,0.000032,0.000000,0.000257,0.000000,0.001320,0.000000,0.000103,0.000048,0.000000,0.000000


In [50]:
mgx.to_csv('mgx_abundance.csv', index=False)

In [8]:
# read in 16S data
amplicon = pd.read_csv("feature-table.csv")
amplicon.rename(columns={'#OTU ID':'taxa'}, inplace=True)


In [9]:
# clean taxonomic names
# keep only genera
amplicon["taxa"] = amplicon['taxa'].str.split("D_5__", expand=True)[1]
amplicon.dropna(inplace=True)


In [10]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
0,Methanobrevibacter,0,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,19,0,0,0
7,Subgroup 10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Subgroup 23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Ilumatobacter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Actinomyces,0,0,7,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
amplicon.sort_values(by = "taxa", ascending=True, inplace=True)

In [12]:
amplicon["taxa"] = amplicon["taxa"].astype(str)

In [13]:
# remove any Nones, uncultured organisms
amplicon = amplicon[~amplicon.taxa.str.contains("uncultured")]
amplicon = amplicon[~amplicon.taxa.str.contains("metagenome")]
amplicon = amplicon[~amplicon.taxa.str.contains("unidentified")]
amplicon = amplicon[~amplicon.taxa.str.contains(r'[0-9]')]
amplicon = amplicon[~amplicon.taxa.str.contains("group")]

In [14]:
# calculate relative abundances
bug_sum = amplicon.sum(axis = 0, skipna = True)
amplicon.iloc[:, 1:] = amplicon.iloc[:, 1:]/bug_sum

In [15]:
amplicon.to_csv('16S_abundance.csv')

### looking for the intersection of taxonomy

In [16]:
# calculate number of taxa in both vs. either one

In [17]:
amplicon_taxa_list = set(amplicon["taxa"])
mgx_taxa_list = set(mgx["taxa"])

In [18]:
intersection = [bug for bug in amplicon_taxa_list if bug in mgx_taxa_list]

def union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

union = union(list(set(amplicon_taxa_list)), list(set(mgx_taxa_list)))


In [19]:
union_df = pd.DataFrame(union)


In [20]:
union_df.head()

,0
0,Eudoraea
1,Planoglabratella opercularis
2,Flavicella
3,Desulfomicrobium
4,Allisonella


In [21]:
union_df.to_csv('taxalist.csv', index=False)

len(intersection)
len(amplicon_taxa_list)-len(intersection)
len(mgx_taxa_list)-len(intersection)

In [22]:
amplicon_only = [bug for bug in amplicon_taxa_list if bug not in intersection]
mgx_only = [bug for bug in mgx_taxa_list if bug not in intersection]


### creating long-form dataframe with data from both sequencing methods

In [23]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
245,Acetanaerobacterium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,Acetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438,Acholeplasma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,Achromobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011094,0.009139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
mgx_unique = [sample for sample in list(mgx.columns.values) if sample not in list(amplicon.columns.values)] 
amplicon_unique = [sample for sample in list(amplicon.columns.values) if sample not in list(mgx.columns.values)]


mgx.drop(mgx_unique, axis=1, inplace=True)
amplicon.drop(amplicon_unique, axis=1, inplace=True)

In [25]:
len(mgx.columns.values)

262

In [26]:
len(amplicon.columns.values)

262

In [27]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [28]:
mgx_melt = pd.melt(mgx, id_vars=["taxa"], var_name = "sampleid", value_name = "mgx_abund")

In [29]:
mgx_melt.head()

,taxa,sampleid,mgx_abund
0,Acidaminococcus,M0652-1F-1A,0.0
1,Acinetobacter,M0652-1F-1A,0.0
2,Actinobacillus,M0652-1F-1A,0.0
3,Actinobaculum,M0652-1F-1A,0.0
4,Actinomyces,M0652-1F-1A,0.0


In [30]:
amplicon_melt.head()

,taxa,sampleid,amplicon_abund
0,Acetanaerobacterium,C0047-7E-1A,0.0
1,Acetobacter,C0047-7E-1A,0.0
2,Acholeplasma,C0047-7E-1A,0.0
3,Achromobacter,C0047-7E-1A,0.0
4,Acidaminococcus,C0047-7E-1A,0.0


In [31]:
merged_taxa = pd.merge(amplicon_melt, mgx_melt, on = ["sampleid", "taxa"], how = "outer")

In [32]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [33]:
merged_taxa.sample(10)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
37530,Granulicatella,M0753-1E-1A,0.0,0.0,0.0,0.0
12111,Rubripirellula,C0694-1E-1A,0.0,NaN,NaN,NaN
10943,Actinomyces,C0683-1E-1A,0.0,0.0,0.0,0.0
58860,Catenibacterium,M1065-1F-1A,0.0,0.0,0.0,0.0
29662,Allisonella,M0699-3F-1A,0.0,NaN,NaN,NaN
12753,Holdemania,C0703-4F-1A,0.0,0.0,0.0,0.0
66924,Epsilon15likevirus,M0715-2E-1A,NaN,0.0,NaN,NaN
75635,Varibaculum,M0958-3F-1A,NaN,0.0,NaN,NaN
77374,Helicobacter,M1059-1F-1A,NaN,0.0,NaN,NaN
50964,Paraprevotella,M0973-1F-1A,0.0,0.0,0.0,0.0


In [34]:
merged_taxa.to_csv('taxa_abundance_comparison.csv')

In [35]:
len(mgx_only)
len(amplicon_only)

59

159

In [36]:
# number of samples where there is one bug picked up by 16S and not mgx

rare_16s = merged_taxa[(merged_taxa.amplicon_abund > 0.0) & (merged_taxa.mgx_abund == 0.0)]
samples_rare_16s = (set(rare_16s["sampleid"]))
num_16s_rare_samples = len(samples_rare_16s)

rare_mgx = merged_taxa[(merged_taxa.mgx_abund > 0.0) & (merged_taxa.amplicon_abund == 0.0)]
samples_rare_mgx = (set(rare_mgx["sampleid"]))
num_mgx_rare_samples = len(samples_rare_mgx)


In [37]:
display(num_16s_rare_samples)
display(num_mgx_rare_samples)

259

258

In [38]:
rare_mgx_taxa = rare_mgx["taxa"]
rare_16s_taxa = rare_16s["taxa"]

In [39]:
len(rare_mgx_taxa)
len(rare_16s_taxa)

923

1456

In [40]:
merged_taxa.head()

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
0,Acetanaerobacterium,C0047-7E-1A,0.0,NaN,NaN,NaN
1,Acetobacter,C0047-7E-1A,0.0,NaN,NaN,NaN
2,Acholeplasma,C0047-7E-1A,0.0,NaN,NaN,NaN
3,Achromobacter,C0047-7E-1A,0.0,NaN,NaN,NaN
4,Acidaminococcus,C0047-7E-1A,0.0,0.0,0.0,0.0


In [41]:
amplicon_avg_abund = merged_taxa.groupby("taxa")["amplicon_abund"].mean()
mgx_avg_abund = merged_taxa.groupby("taxa")["mgx_abund"].mean()
taxa_list = sorted(set(merged_taxa["taxa"]))

In [42]:
mean_taxa_abund = pd.DataFrame(
    (zip(taxa_list, amplicon_avg_abund, mgx_avg_abund)),  
    columns = ['taxa','amp_avg_abund', 'mgx_avg_abund'])

In [43]:
mean_taxa_abund["abs_diff"] = abs(mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"])
mean_taxa_abund["total_diff"] = mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"]


In [44]:
mean_taxa_abund.sort_values("abs_diff", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [45]:
mean_taxa_abund.sample(10)

,taxa,amp_avg_abund,mgx_avg_abund,abs_diff,total_diff
157,Labrenzia,0.000018,NaN,NaN,NaN
274,Streptococcus,0.017243,0.013967,0.003276,0.003276
236,Pseudoalteromonas,0.000137,NaN,NaN,NaN
48,Butyricimonas,0.002619,0.000004,0.002615,0.002615
112,Facklamia,NaN,0.000000,NaN,NaN
165,Lentimicrobium,0.000000,NaN,NaN,NaN
60,Catabacter,0.000010,NaN,NaN,NaN
128,Gaetbulibacter,0.000011,NaN,NaN,NaN
154,Joostella,0.000019,NaN,NaN,NaN
42,Blastopirellula,0.000053,NaN,NaN,NaN


In [46]:
mean_taxa_abund.to_csv('taxa_difference.csv')